In [1]:
import numpy as np
import keras
from keras.layers import Input, Dense, Activation, BatchNormalization, Reshape, UpSampling2D, Conv2D, MaxPool2D, Flatten

class Generator(object):
    def __init__(self, latent_dim):
        generator_input = keras.Input(shape=(latent_dim,))
        x = Dense(1024)(generator_input)
        x = Activation('tanh')(x)
        x = Dense(128*7*7)(x)
        x = BatchNormalization()(x)
        x = Activation('tanh')(x)
        x = Reshape((7, 7, 128))(x)
        x = UpSampling2D(size=(2, 2))(x)
        x = Conv2D(64, 5, padding='same')(x)
        x = Activation('tanh')(x)
        x = UpSampling2D(size=(2, 2))(x)
        x = Conv2D(1, 5, padding='same')(x)
        x = Activation('tanh')(x)
        self.generator = keras.models.Model(generator_input, x)
    def get_model(self):
        return self.generator


class Discriminator(object):
    def __init__(self, height, width, channels):
        discriminator_input = Input(shape=(height, width, channels))
        x = Conv2D(64, 5, padding='same')(discriminator_input)
        x = Activation('tanh')(x)
        x = MaxPool2D()(x)
        x = Conv2D(128, 5)(x)
        x = Activation('tanh')(x)
        x = MaxPool2D()(x)
        x = Flatten()(x)
        x = Dense(1024)(x)
        x = Activation('tanh')(x)
        x = Dense(1, activation='sigmoid')(x)
        self.discriminator = keras.models.Model(discriminator_input, x)

    def get_model(self):
        return self.discriminator


class DCGAN(object):
    def __init__(self, latent_dim, height, width, channels):
        # set generator
        self._latent_dim = latent_dim
        g = Generator(latent_dim)
        self._generator = g.get_model()
        # set discriminator
        d = Discriminator(height, width, channels)
        self._discriminator = d.get_model()
        # compile discriminator
        discriminator_optimizer = keras.optimizers.SGD(lr=0.0005, momentum=0.9, nesterov=True)
        self._discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')
        # disable training when combined with generator
        self._discriminator.trainable = False
        # set DCGAN
        dcgan_input = keras.Input(shape=(latent_dim,))
        dcgan_output = self._discriminator(self._generator(dcgan_input))
        self._dcgan = keras.models.Model(dcgan_input, dcgan_output)
        # compile DCGAN
        dcgan_optimizer = keras.optimizers.SGD(lr=0.0005, momentum=0.9, nesterov=True)
        self._dcgan.compile(optimizer=dcgan_optimizer, loss='binary_crossentropy')

    def train(self, real_images, batch_size):
        # Train so discriminator can detect fake
        random_latent_vectors = np.random.normal(size=(batch_size, self._latent_dim))
        generated_images = self._generator.predict(random_latent_vectors)
        labels = np.ones((batch_size, 1))
        labels += 0.05 * np.random.random(labels.shape)
        d_loss1 = self._discriminator.train_on_batch(generated_images, labels)
        # Train so discriminator can detect real
        labels = np.zeros((batch_size, 1))
        labels += 0.05 * np.random.random(labels.shape)
        d_loss2 = self._discriminator.train_on_batch(real_images, labels)
        d_loss = (d_loss1 + d_loss2)/2.0
        # Train so generator can fool discriminator
        random_latent_vectors = np.random.normal(size=(batch_size, self._latent_dim))
        misleading_targets = np.zeros((batch_size, 1))
        g_loss = self._dcgan.train_on_batch(random_latent_vectors, misleading_targets)
        return d_loss, g_loss

    def predict(self, latent_vector):
        return self._generator.predict(latent_vector)

    def load_weights(self, file_path, by_name=False):
        self._dcgan.load_weights(file_path, by_name)

    def save_weights(self, file_path, overwrite=True):
        self._dcgan.save_weights(file_path, overwrite)

Using TensorFlow backend.


In [2]:
import os
import numpy as np
import keras
from keras.preprocessing import image
from PIL import Image

# Normalize image from 0 - 255 to -1 - 1
def normalize(X):
    return (X - 127.5)/127.5

# Denormalize from -1 - 1 to 0 - 255
def denormalize(X):
    return (X + 1.0)*127.5

def train(latent_dim, height, width, channels,file_path, X_train, Y_train): 
    X_train = X_train.reshape((X_train.shape[0],) + (height, width, channels)).astype('float32')
    X_train = normalize(X_train)
    epochs = 20
    batch_size = 128
    iterations = X_train.shape[0]//batch_size
    dcgan = DCGAN(latent_dim, height, width, channels)
    #dcgan.load_weights(file_path+'gan_epoch20.h5')
    for epoch in range(epochs):
        for iteration in range(iterations):
            real_images = X_train[iteration*batch_size:(iteration+1)*batch_size]
            d_loss, g_loss = dcgan.train(real_images, batch_size)
            if (iteration + 1)%10 == 0:
                print('discriminator loss:', d_loss)
                print('generator loss:', g_loss)
                print(iteration)
                with open('loss.txt', 'a') as f:
                    f.write(str(d_loss) + ',' + str(g_loss) + '\r')
        dcgan.save_weights(file_path+'gan' + '_epoch' + str(epoch + 1) + '.h5')
        print('epoch' + str(epoch) + ' end')
        print()

def predict(latent_dim, height, width, channels,file_path, epoch):
    np.random.seed(seed=32)
    random_latent_vectors = np.random.normal(size=(100, latent_dim))
    dcgan = DCGAN(latent_dim, height, width, channels)
    dcgan.load_weights(file_path+'gan_epoch'+str(epoch)+'.h5')
    generated_images = dcgan.predict(random_latent_vectors)
    
    for i, generated_image in enumerate(generated_images):
        img = image.array_to_img(denormalize(generated_image), scale=False)
        img.save(os.path.join(file_path+ str(i) + '.png'))
        
    canvas = Image.new('RGB', ((width+1)*10, (height+1)*10), (255, 255, 255))
    for i in range(100):
        xpng = Image.open(file_path+ str(i) + '.png', 'r')
        canvas.paste(xpng, (i%10*(width+1), i//10*(height+1)))
    canvas.save(file_path+'images'+str(epoch)+'.jpg', 'JPEG', quality=100, optimize=True)
    



In [3]:
latent_dim = 100
height = 28
width = 28
channels = 1
file_path = "/root/userspace/private/gan/generated_mnist_five5000/"


In [8]:
(cifar_X , cifar_y), (_, _) = keras.datasets.mnist.load_data()
X_train=[]
Y_train=[]
for x in range(cifar_X.shape[0]):
    if cifar_y[x]==5:
        X_train.append(cifar_X[x])
        Y_train.append(cifar_y[x])
X_train=np.array(X_train)
Y_train=np.array(Y_train)

print(X_train[0])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   93  82  82  56  39   0   0   0   0   0]
 [  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 18

In [5]:
train(latent_dim, height, width, channels, file_path, X_train, Y_train)

discriminator loss: 0.486968100071
generator loss: 0.603532
9
discriminator loss: 0.432384312153
generator loss: 0.808388
19
discriminator loss: 0.412638306618
generator loss: 1.08217
29
discriminator loss: 0.454953372478
generator loss: 1.30373
39
epoch0 end

discriminator loss: 0.388015836477
generator loss: 1.45004
9
discriminator loss: 0.232709467411
generator loss: 1.62527
19
discriminator loss: 0.190990030766
generator loss: 1.69842
29
discriminator loss: 0.122863985598
generator loss: 1.77268
39
epoch1 end

discriminator loss: 0.175120636821
generator loss: 1.48031
9
discriminator loss: 0.205781936646
generator loss: 1.35587
19
discriminator loss: 0.300023525953
generator loss: 1.43802
29
discriminator loss: 0.166272506118
generator loss: 2.2323
39
epoch2 end

discriminator loss: 0.168191671371
generator loss: 2.79407
9
discriminator loss: 0.231197848916
generator loss: 2.54811
19
discriminator loss: 0.308319985867
generator loss: 1.75586
29
discriminator loss: 0.186249509454
ge

In [6]:
for i in range(1,21,1):
    predict(latent_dim, height, width, channels,file_path, epoch=i)